In [ ]:
1. RNN



# Import necessary libraries
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.datasets import imdb
from random import randint

# Load the IMDB dataset, limiting to the top 5000 most frequent words
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

# Pad sequences to ensure uniform input length
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

# Create the LSTM model
model = Sequential()
model.add(Embedding(5000, 128))          # Embedding layer for word embeddings
model.add(LSTM(128, activation='tanh', recurrent_activation='sigmoid'))  # LSTM layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification


# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_test, y_test), shuffle=True, verbose=1)

# Print model summary
model.summary()




# Predict on the test set
predictions = model.predict(x_test)

# Randomly select a review from the test set for evaluation
arr_ind = randint(0, len(x_test) - 1)  # Random index
index = imdb.get_word_index()           # Word index dictionary
reverse_index = dict([(value, key) for (key, value) in index.items()])  # Reverse dictionary for decoding
decoded = " ".join([reverse_index.get(i - 3, "#") for i in x_test[arr_ind]])  # Decode review text

# Determine the sentiment of the review
predicted_sentiment = "Positive" if predictions[arr_ind] >= 0.5 else "Negative"
actual_sentiment = "Positive" if y_test[arr_ind] == 1 else "Negative"

# Print results
print("Sentence:", decoded)
print("Review:", predicted_sentiment)
print("Predicted Value:", predictions[arr_ind][0])
print("Expected Value:", y_test[arr_ind])



In [ ]:
2. autoencoder

import numpy as np
import keras
from keras import layers
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

input_img = keras.Input(shape=(784,))
encoded = layers.Dense(128, activation='sigmoid')(input_img)
decoded = layers.Dense(784, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

import matplotlib.pyplot as plt

encoded_imgs = autoencoder.predict(x_test)
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()



# ADDING SPARISTY

from keras import regularizers

encoding_dim = 32

input_img = keras.Input(shape=(784,))
# Add a Dense layer with a L1 activity regularizer
encoded = layers.Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = layers.Dense(784, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_img, decoded)

# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

autoencoder.fit(x_train, x_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# Use Matplotlib
import matplotlib.pyplot as plt

n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()




#DENOISING AUTOENCODER

#Denoising

from tensorflow.keras.layers import GaussianNoise

(x_train,_),(x_test,_) = mnist.load_data()
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
print(x_train.shape)


x_train=x_train.reshape((len(x_train),np.prod(x_train.shape[1:])))
x_test=x_test.reshape((len(x_test),np.prod(x_test.shape[1:])))
print(x_train.shape)

#Architechture

input_img = Input(shape=(784,))
#add noise to input
noisy_input = GaussianNoise(0.5)(input_img)

encoded = Dense(128,activation='sigmoid')(input_img)
decoded = Dense(784,activation='sigmoid')(encoded)

#Build autencoder model

autoencoder = Model(input_img,decoded)
#compile
autoencoder.compile(optimizer= Adam(),loss='binary_crossentropy')
#train
autoencoder.fit(x_train,x_train,epochs=5,batch_size=256,shuffle=True,validation_data=(x_test,x_test))


import matplotlib.pyplot as plt

#encode and decode some digits
encoded_imgs = autoencoder.predict(x_test)

# Generate noisy images
noise_factor = 0.5
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

n = 10  # How many digits we will display
plt.figure(figsize=(20, 6))
for i in range(n):
    # Display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display noisy
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(encoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
3. optimizers


import numpy as np
import matplotlib.pyplot as plt
#generate synthetic data
np.random.seed(42)
X=2*np.random.rand(100,1)
y=4+3*X+np.random.randn(100,1)

#Add bias term (intercept)
X_b=np.c_[np.ones((100,1)),X]


# TO PERFORM BATCH GRADIENT DESCENT

def batch_gradient_descent(X,y,learning_rate=0.1,n_iterations=1000):      #Function to perform batch Gradient Descent
  m=len(y)
  theta=np.random.randn(2,1)
  print(theta)
  for iteration in range(n_iterations):
    gradients=2/m*X.T.dot(X.dot(theta)-y)
    theta=theta-learning_rate*gradients
  return theta

theta_bgd=batch_gradient_descent(X_b,y);
print("BGD Theta:", theta_bgd)



#functions to perform stochastic gradient descent
def stochastic_gradient_descent(X,y,learning_rate=0.1,n_iterations=50):
  m=len(y)
  theta=np.random.randn(2,1)
  for epoch in range(n_epochs):
    for i in range(m):
      random_index=np.random.randint(m)
      xi=X[random_index:random_index+1]
      yi=y[random_index:random_index+1]
      gradients=2*xi.T.dot(xi.dot(theta)-yi)
      theta=theta-learning _rate*gradients
      return theta

theta_sgd=stochastic_gradient_descent(X_b,y)
print("SGD Theta:",)


# to perform Mini-batch Gradient Descent
def mini_batch_gradient_descent(X,y,learning_rate=0.1,n_iterations=50,batch_size=20):
  m=len(y)
  theta=np.random.randn(2,1)
  for iteration in range(n_iterations):
    shuffled_indices=np.random.permutation(m)
    X_shuffled=X[shuffled_indices]
    y_shuffled=y[shuffled_indices]
    for i in range(0,m,batch_size):
      xi=X_shuffled[i:i+batch_size]
      yi=y_shuffled[i:i+batch_size]
      gradients=2/len(xi)*xi.T.dot(xi.dot(theta)-yi)
      theta=theta-learning_rate*gradients
      return theta

theta_mbgd=mini_batch_gradient_descent(X_b,y)
print("Mini-batch GD Theta:",theta_mbgd)


#plotting the result

import matplotlib.pyplot as plt

def plot_gradient_descent(X,y,theta_bgd,theta_sgd,theta_mbgd):
  plt.plot(X,y,"b.")
  X_new = np.array([[0],[2]])
  X_new_b = np.c_[np.ones((2,1)),X_new]
  print(X_new)
  y_predict_bgd = X_new_b.dot(theta_bgd)

  y_predict_sgd = X_new_b.dot(theta_sgd)
  y_predict_mbgd = X_new_b.dot(theta_mbgd)
  plt.plot(X_new,y_predict_bgd,"r-",linewidth = 4,label="BGD")
  plt.plot(X_new,y_predict_sgd,"g-",linewidth = 2,label="SGD")
  plt.plot(X_new,y_predict_mbgd,"y-",linewidth=2,label="MBGD")
  plt.xlabel("$x_1$")
  plt.ylabel("$y$")
  plt.legend(loc="upper left")
  plt.title("Gradient DEscent Comparison")
  plt.show()

plot_gradient_descent(X,y,theta_bgd,theta_sgd,theta_mbgd)


##Momentum based Gradient Descent

def gradient_descent_with_momentum(X , y ,theta , learning_rate , gamma , num_iteration):
    m=len(y)
    velocity = np.zeros_like(theta)

    for i in range(num_iteration):
        gradient=(1/m) * X.T.dot(X.dot(theta)-y)
        velocity = gamma*velocity + learning_rate * gradient
        theta = theta - velocity
    return theta

X = np.array([[1,2],[3,4] , [5,6]])
y = np.array([1,2,3])
theta = np.zeros(X.shape[1])
print(theta)
learning_rate = 0.1
gamma = 0.9
num_iterations=1000

theta = gradient_descent_with_momentum(X , y , theta , learning_rate , gamma , num_iterations)
print(theta)


#for adagard
#for Adagrad
import numpy as np
import matplotlib.pyplot as plt

#Generate synthetic data
np.random.seed(42)
X=2*np.random.rand(100,1)
y=4+3*X+np.random.randn(100,1)

#Add bias term (x0=1)to each instance
X_b=np.c_[np.ones((100,1)),X]

#Parameters for Adagrad
learning_rate=0.1
epsilon=1e-8
num_iterations=1000

#Initial parameter vector(theta)
theta=np.random.randn(2,1)

#Adagrad specific parameters
gradient_accum=np.zeros((2,1))

#Adagrad optimizer
for iteration in range(num_iterations):
  gradients=2/len(X_b)*X_b.T.dot(X_b.dot(theta)-y)
  gradient_accum += gradients**2
  adjusted_gradients=gradients/(np.sqrt(gradient_accum)+epsilon)
  theta-=learning_rate*adjusted_gradients

print("Optimized parameters using Adagrad:",theta)

#plot the result
plt.scatter(X,y)
plt.plot(X,X_b.dot(theta),color='red',linewidth=2,label='Adagrad')
plt.xlabel('X')
plt.xlabel('y')
plt.legend()
plt.show()



#For RMS
For RMS
import numpy as np
import matplotlib.pyplot as plt

# Generate synthetic data
np.random.seed(42)  # For reproducibility
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)
X_b = np.c_[np.ones((100, 1)), X]  # Add bias term

# Hyperparameters
learning_rate = 0.01
epsilon = 1e-8
beta = 0.9
num_iterations = 1000

# Initialize parameters
theta = np.random.randn(2, 1)
s = np.zeros((2, 1))  # Initialize the running average of squared gradients

# RMSprop optimization
for iteration in range(num_iterations):
    gradients = 2 / len(X_b) * X_b.T.dot(X_b.dot(theta) - y)
    s = beta * s + (1 - beta) * gradients ** 2  # Update the running average of squared gradients
    adjusted_gradients = gradients / (np.sqrt(s) + epsilon)
    theta -= learning_rate * adjusted_gradients

    if iteration % 100 == 0:
        print(f"Iteration {iteration}: theta = {theta.ravel()}")

print("Theta:", theta)

# Plotting the results
plt.scatter(X, y, label='Data')
plt.plot(X, X_b.dot(theta), color='red', linewidth=2, label='RMSprop')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

In [ ]:
4. CNN USING FILTERS

import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imshow, imread
from skimage.color import rgb2yuv, rgb2hsv, rgb2gray, yuv2rgb, hsv2rgb
from scipy.signal import convolve2d

#Reading Image
dog = imread('/content/sample_data/st,small,507x507-pad,600x600,f8f8f8.jpg')
plt.figure(num=None, figsize=(8,6), dpi=80)
imshow(dog)


#Filter MAtrices

sharpen = np.array([[0,-1,0],
                    [-1,5,-1],
                    [0,-1,0]])

blur = np.array([[0.11,0.11,0.11],
                    [0.11,0.11,0.11],
                    [0.11,0.11,0.11]])


vertical = np.array([[-1,0,1],
                    [-2,0,2],
                    [-1,0,1]])


gaussian = (1/16.0) * np.array([[1,2,1],
                                [2,4,2],
                                [1,2,1]])



#plotting the filters

fig,ax = plt.subplots(1,3, figsize = (17,10))
ax[0].imshow(sharpen, cmap='gray')
ax[0].set_title(f'Sharpen', fontsize=18)

ax[1].imshow(blur, cmap='gray')
ax[1].set_title(f'Blur', fontsize=18)

ax[2].imshow(vertical, cmap='gray')
ax[2].set_title(f'Vertical', fontsize=18)


#Grayscaling Image
dog_gray = rgb2gray(dog)
plt.figure(num=None, figsize=(8,6), dpi=80)
imshow(dog_gray)


#Function for applying filters
def multi_convolver(image, kernel, iterations):
  for i in range(iterations):
    image = convolve2d(image, kernel, 'same', boundary = 'fill', fillvalue = 0)
  return image

convolved_image = multi_convolver(dog_gray, sharpen, 1)

plt.figure(num=None, figsize=(8,6), dpi=80)
imshow(convolved_image);



#For colored Image
def convolver_rgb(image, kernel, iterations = 1):
  convolved_image_r = multi_convolver(image[:,:,0], kernel, iterations)
  convolved_image_g = multi_convolver(image[:,:,1], kernel, iterations)
  convolved_image_b = multi_convolver(image[:,:,2], kernel, iterations)

  reformed_image = np.dstack((np.rint(abs(convolved_image_r)),np.rint(abs(convolved_image_g)),np.rint(abs(convolved_image_b))))/255

  fig,ax = plt.subplots(1,3, figsize = (17,10))

  ax[0].imshow(abs(convolved_image_r), cmap='Reds')
  ax[0].set_title(f'Red', fontsize=15)

  ax[1].imshow(abs(convolved_image_g), cmap='Greens')
  ax[1].set_title(f'Green', fontsize=18)

  ax[2].imshow(abs(convolved_image_b), cmap='Blues')
  ax[2].set_title(f'Blue', fontsize=18)

  return np.array(reformed_image*255).astype(np.uint8)

#Can add different filters (defined above) here
convolved_rgb_gauss = convolver_rgb(dog, vertical.T ,1)




#CNN PARAMETER CALCULATION
from keras.models import Sequential
from keras.layers import Conv2D


model = Sequential()
model.add(Conv2D(32, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 padding='same',
                 use_bias=False))
model.add(Conv2D(17, (3,3), padding='same', use_bias=False))
model.add(Conv2D(13, (3,3), padding='same', use_bias=False))
model.add(Conv2D(7, (3,3), padding='same', use_bias=False))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()


model = Sequential()
model.add(Conv2D(32, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 padding='same',
                 use_bias=True))
model.add(Conv2D(17, (3,3), padding='same', use_bias=True))
model.add(Conv2D(13, (3,3), padding='same', use_bias=True))
model.add(Conv2D(7, (3,3), padding='same', use_bias=True))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()


model = Sequential()
model.add(Conv2D(32, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 use_bias=False))
model.add(Conv2D(17, (3,3), use_bias=False))
model.add(Conv2D(13, (3,3), use_bias=False))
model.add(Conv2D(7, (3,3), use_bias=False))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()



model = Sequential()
model.add(Conv2D(32, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 padding='valid',
                 use_bias=False))
model.add(Conv2D(17, (3,3), padding='valid', use_bias=False))
model.add(Conv2D(13, (3,3), padding='valid', use_bias=False))
model.add(Conv2D(7, (3,3), padding='valid', use_bias=False))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()


model = Sequential()
model.add(Conv2D(10, input_shape=(28,28,3),
                 kernel_size = (5,5),
                 strides = (1,1),
                 padding='valid',
                 use_bias=False))
model.add(Conv2D(20, (5,5), (2,2), padding='valid', use_bias=False))
model.add(Conv2D(40, (5,5), (2,2), padding='valid', use_bias=False))


model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()



model = Sequential()
model.add(Conv2D(10, input_shape=(39,39,3),
                 kernel_size = (3,3),
                 padding='valid',))
model.add(Conv2D(20, (5,5), (2,2), padding='valid' ))
model.add(Conv2D(40, (5,5), (2,2),padding='valid'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()




# MAX POLLING
import tensorflow as tf
x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.],
                 [7., 8., 9.]])
x = tf.reshape(x, [1, 3, 3, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')
max_pool_2d(x)



x = tf.constant([[1., 2., 3., 4.],
                 [5., 6., 7., 8.],
                 [9., 10., 11., 12.]])
x = tf.reshape(x, [1, 3, 4, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
   strides=(2, 2), padding='valid')
max_pool_2d(x)


x = tf.constant([[1., 2., 3., 4.],
                 [5., 6., 7., 8.],
                 [9., 10., 11., 12.]])
x = tf.reshape(x, [1, 3, 4, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
   strides=(2, 2), padding='same')
max_pool_2d(x)


x = tf.constant([[1., 2., 3., 4.],
                 [5., 6., 7., 8.],
                 [9., 10., 11., 12.]])
x = tf.reshape(x, [1, 3, 4, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
   strides=(1, 1), padding='same')
max_pool_2d(x)

In [ ]:
5. REGULARIZATION

import tensorflow as tf
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
import warnings
from mlxtend.plotting import plot_decision_regions
from matplotlib.colors import ListedColormap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles,make_moons



np.random.seed(800)
X,y=make_moons(n_samples=100, noise=0.2, random_state=1)
zero_one_colourmap=ListedColormap(('red','green'))
rcParams['figure.figsize']=14,7
plt.scatter(X[:,0],X[:,1],
c=y, s=100,
cmap=zero_one_colourmap)
plt.show()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = Sequential()
model.add(Dense(500, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=1000, verbose=0)

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

reg_model = Sequential()
reg_model.add(Dense(500, input_dim=2, activation='relu', kernel_regularizer='l2'))
reg_model.add(Dense(1, activation='sigmoid', kernel_regularizer='l2'))
reg_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

reg_history = reg_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, verbose=0)

plt.plot(reg_history.history['loss'], label='train')
plt.plot(reg_history.history['val_loss'], label='test')
plt.legend()
plt.show()




In [ ]:
6. AUGUMENTION

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255  # Reshape and normalize
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical", input_shape=(28, 28, 1)),  # Specify input shape here
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])

# Create a simple CNN model
def create_cnn_model():
    model = models.Sequential()
    model.add(data_augmentation)  # Include data augmentation in the model
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer

    return model

# Create and compile the model
model = create_cnn_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with data augmentation
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='test accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
7. DROP OUT

# DropOut with CNN on Mnist dataset

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Build CNN model with Dropout
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout layer with 50% rate
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

In [ ]:
8. CNN ON MINIST DATASET USING FILTERS

# Filter on Mnist using CNN

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255  # Reshape and normalize
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Define the CNN model
def create_cnn_model():
    model = models.Sequential()

    # Convolutional layer with 3x3 filters (Sharpening Filter)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

    # Convolutional layer with 5x5 filters (Blur Filter)
    model.add(layers.Conv2D(64, (5, 5), activation='relu'))

    # Pooling layer
    model.add(layers.MaxPooling2D((2, 2)))

    # Another convolutional layer (Edge Detection Filter)
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))

    # Flattening layer
    model.add(layers.Flatten())

    # Fully connected layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.Dense(10, activation='softmax'))  # Output layer

    return model

# Create and compile the model
model = create_cnn_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='test accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()